# Section 1: Installing and Importing Needed Packages

In [ ]:
from google.colab import drive

!pip3 install boto3
!pip install pandasql

In [ ]:
import numpy as np 
import json
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os
import pandasql as ps #SQL on Pandas Dataframe
from pandasql import sqldf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
!apt install libkrb5-dev
!wget https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install findspark
!pip install sparkmagic
!pip install pyspark
!pip install pyspark --user
!pip install seaborn --user
!pip install plotly --user
!pip install imageio --user
!pip install folium --user

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libkrb5-dev is already the newest version (1.16-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
--2022-04-12 18:41:20--  https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving www-us.apache.org (www-us.apache.org)... failed: Name or service not known.
wget: unable to resolve host address ‘www-us.apache.org’
tar: spark-2.4.5-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!apt update
!apt install gcc python-dev libkrb5-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://archive.ubun

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkFiles

import os

spark = SparkSession.builder.appName('mcit550-final_project').getOrCreate()

# Section 2: Importing Raw Data Files and Processing

Sources: 

*   https://www.imdb.com/interfaces/

---






In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Getting Basic Title data from Google Drive folder

title_basics = open("/content/gdrive/MyDrive/IMDb_Dataset/title_basics.tsv")
title_basics_df = pd.read_csv(title_basics, sep='\t', na_values='\\N',header=0)
#title_basics_df.head()
#title_basics_df.dtypes
#title_basics_df.describe()

#Getting the distinct values of titleType
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT distinct titleType 
       FROM title_basics_df 
       ORDER BY titleType;"""

types = pysqldf(q)
types



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,titleType
0,movie
1,short
2,tvEpisode
3,tvMiniSeries
4,tvMovie
5,tvPilot
6,tvSeries
7,tvShort
8,tvSpecial
9,video


In [ ]:
#Select only tvSeries and then use these to truncate all other tables

q = """SELECT * 
       FROM title_basics_df 
       WHERE titleType = "tvSeries";"""

title_basics_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
#title_basics_df_2.head(20)
title_basics_df_2.describe()

,isAdult,startYear,endYear
count,220998.000000,205022.000000,69222.000000
mean,0.009186,2005.642141,2003.017206
std,0.095401,15.815716,17.033259
min,0.000000,1906.000000,1925.000000
25%,0.000000,2001.000000,1995.000000
50%,0.000000,2011.000000,2009.000000
75%,0.000000,2017.000000,2016.000000
max,1.000000,2026.000000,2027.000000


In [ ]:
#Getting Basic Title into Titles (dropping genres)

Titles_df = title_basics_df_2.rename(columns={'tconst':'tid','titleType':'type','primaryTitle':'pTitle','originalTitle':'oTitle'})
Titles_df = Titles_df.drop(columns={'genres'})
Titles_df.head()

Titles_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Titles.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
#From Basic Title, exploding out Genres

Genres_df = title_basics_df_2[['tconst','genres']]
Genres_df = Genres_df.rename(columns={'tconst':'tid','genres':'genre'})

Genres_df = Genres_df.dropna()

Genres_df = Genres_df.assign(genre=Genres_df.genre.str.split(',')).explode('genre').reset_index(drop=True)

Genres_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Genres.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
# Getting Title Ratings data from Google Drive folder

title_ratings = open("/content/gdrive/MyDrive/IMDb_Dataset/title_ratings.tsv")
title_ratings_df = pd.read_csv(title_ratings, sep='\t', na_values='\\N',header=0)
title_ratings_df.head()
#title_ratings_df.describe()



,tconst,averageRating,numVotes
0,tt0000001,5.7,1858
1,tt0000002,6.0,243
2,tt0000003,6.5,1628
3,tt0000004,6.0,158
4,tt0000005,6.2,2455


In [ ]:
#Truncate list to tvSeries
q = """SELECT r.tconst AS tconst, averageRating, numVotes
       FROM title_basics_df_2 t JOIN title_ratings_df r ON t.tconst = r.tconst ;"""

title_ratings_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
title_ratings_df_2.head(20)
title_ratings_df_2.describe()

,averageRating,numVotes
count,80097.000000,8.009700e+04
mean,6.817160,1.349065e+03
std,1.473883,1.718550e+04
min,1.000000,5.000000e+00
25%,6.100000,1.300000e+01
50%,7.100000,3.100000e+01
75%,7.900000,1.280000e+02
max,10.000000,1.950066e+06


In [ ]:
#Cleaning Title Ratings into Ratings

Ratings_df = title_ratings_df_2.rename(columns={'tconst':'tid','averageRating':'aveRating','numVotes':'numvotes'})
Ratings_df.head()

Ratings_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Ratings.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
# Getting Episode Title data from Google Drive folder

title_episode = open("/content/gdrive/MyDrive/IMDb_Dataset/title_episode.tsv")
title_episode_df = pd.read_csv(title_episode, sep='\t', na_values='\\N',header=0)
title_episode_df.head()
#title_episode_df.describe()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0020666,tt15180956,1.0,2.0
1,tt0020829,tt15180956,1.0,1.0
2,tt0021166,tt15180956,1.0,3.0
3,tt0021612,tt15180956,2.0,2.0
4,tt0021655,tt15180956,2.0,5.0


In [ ]:
#Truncate list to tvEpisodes in the titles dataframe

q = """SELECT *
      FROM title_episode_df
      WHERE parentTconst IN (SELECT DISTINCT tconst FROM title_basics_df_2);"""

title_episode_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
title_episode_df_2.head(20)
title_episode_df_2.describe()

,seasonNumber,episodeNumber
count,4.956505e+06,4.956505e+06
mean,3.857930e+00,3.926901e+02
std,2.481223e+01,1.305771e+03
min,1.000000e+00,0.000000e+00
25%,1.000000e+00,7.000000e+00
50%,1.000000e+00,2.600000e+01
75%,3.000000e+00,1.410000e+02
max,2.021000e+03,9.133400e+04


In [ ]:
#Getting Episode Title into EpisodeTitles

EpisodeTitles_df = title_episode_df_2.rename(columns={'tconst':'episodeId','parentTconst':'parentId',
                                                    'seasonNumber':'seasonNum','episodeNumber':'episodeNum'})
EpisodeTitles_df.head()

EpisodeTitles_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/EpisodeTitles.tsv',index=False,na_rep=r'\N',sep='\t')


In [ ]:
# Getting Crew Title data from Google Drive folder

title_crew = open("/content/gdrive/MyDrive/IMDb_Dataset/title_crew.tsv")
title_crew_df = pd.read_csv(title_crew, sep='\t', na_values='\\N',header=0)
#title_crew_df.head()
title_crew_df.describe()

,tconst,directors,writers
count,8691512,4955911,4439253
unique,8691512,841344,1163958
top,tt0000001,nm1203430,nm6352729
freq,1,10955,12089


In [ ]:
#Truncate list to tvSeries in the titles dataframe

q = """SELECT r.tconst AS tconst, directors, writers
       FROM title_basics_df_2 t JOIN title_crew_df r ON t.tconst = r.tconst ;"""

title_crew_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
title_crew_df_2.head(20)
title_crew_df_2.describe()

,tconst,directors,writers
count,220998,155937,134508
unique,220998,116107,114429
top,tt0025509,nm11365048,nm11365048
freq,1,129,105


In [ ]:
#Getting Crew Title into Directors
Directors_df = title_crew_df_2[['tconst','directors']]
Directors_df = Directors_df.rename(columns={'tconst':'tid','directors':'pid'})
Directors_df = Directors_df.dropna()
Directors_df = Directors_df.assign(pid=Directors_df.pid.str.split(',')).explode('pid').reset_index(drop=True)
Directors_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Directors.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
#Getting Crew Title into Writers
Writers_df = title_crew_df_2[['tconst','writers']]
Writers_df = Writers_df.rename(columns={'tconst':'tid','writers':'pid'})
Writers_df = Writers_df.dropna()
Writers_df = Writers_df.assign(pid=Writers_df.pid.str.split(',')).explode('pid').reset_index(drop=True)
Writers_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Writers.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
# Getting Title Akas data from Google Drive folder

title_akas = open("/content/gdrive/MyDrive/IMDb_Dataset/title_akas.tsv")
title_akas_df = pd.read_csv(title_akas, sep='\t', na_values='\\N',header=0)
title_akas_df.head()
#title_akas_df.describe()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0.0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0.0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0.0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0.0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0.0


In [ ]:
#Truncate list to tvSeries in the titles dataframe

q = """SELECT *
      FROM title_akas_df
      WHERE titleId IN (SELECT DISTINCT tconst FROM title_basics_df_2);"""

title_akas_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
title_akas_df_2.head(20)
title_akas_df_2.describe()

,ordering,isOriginalTitle
count,621579.000000,621560.000000
mean,4.325893,0.293124
std,6.348385,0.455195
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,4.000000,1.000000
max,177.000000,1.000000


In [ ]:
#Getting Title Akas into AdditionalTitles
AdditionalTitles_df = title_akas_df_2.rename(columns={'titleId':'tid','isOriginalTitle':'isOriginal'})
AdditionalTitles_df = AdditionalTitles_df.drop(columns={'attributes','types'})
AdditionalTitles_df.head()

AdditionalTitles_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/AdditionalTitles.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
# Getting Principal Title data from Google Drive folder

title_principal = open("/content/gdrive/MyDrive/IMDb_Dataset/title_principals.tsv")
title_principal_df = pd.read_csv(title_principal, sep='\t', na_values='\\N',header=0)
title_principal_df.head()
#title_principal_df.describe()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [ ]:
#Cleaning Principal Title into PrincipalTitle

PrincipalTitle_df = title_principal_df.drop(columns={'characters'})
PrincipalTitle_df = PrincipalTitle_df.rename(columns={'tconst':'tid','nconst':'pid'})
PrincipalTitle_df.head()
#PrincipalTitle_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/PrincipalTitle.tsv',index=False,na_rep=r'\N',sep='\t')

,tid,ordering,pid,category,job
0,tt0000001,1,nm1588970,self,NaN
1,tt0000001,2,nm0005690,director,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography
3,tt0000002,1,nm0721526,director,NaN
4,tt0000002,2,nm1335271,composer,NaN


In [ ]:
#Truncate list to tvSeries in the titles dataframe

q = """SELECT r.tid AS tid, ordering, pid, category, job
       FROM title_basics_df_2 t JOIN PrincipalTitle_df r ON t.tconst = r.tid
      ;"""

PrincipalTitle_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe and output PrincipalTitle
PrincipalTitle_df_2.head(20)
#PrincipalTitle_df_2.describe()
#PrincipalTitle_df_2.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/PrincipalTitle.tsv',index=False,na_rep=r'\N',sep='\t')

,tid,ordering,pid,category,job
0,tt0025509,1,nm0062428,actor,None
1,tt0025509,2,nm0889024,actor,None
2,tt0025509,3,nm0044139,actor,None
3,tt0025509,4,nm0213172,actor,None
4,tt0025509,5,nm0241348,actor,None
5,tt0025509,6,nm0312812,actor,None
6,tt0025509,7,nm0470307,actor,None
7,tt0025509,8,nm0560915,actor,None
8,tt0025509,9,nm0622069,actor,None
9,tt0025509,10,nm0785771,actor,None


In [ ]:
#Cleaning Principal Title into Cast_In

Cast_In_df = title_principal_df[['tconst','nconst','characters']]
Cast_In_df = Cast_In_df.rename(columns={'tconst':'tid','nconst':'pid','characters':'character'})
Cast_In_df = Cast_In_df.dropna()
Cast_In_df.character = Cast_In_df.character.str.replace('[\"\[\]]','',regex=True)
Cast_In_df.character = Cast_In_df.character.str.replace('\\','|',regex=True)
Cast_In_df = Cast_In_df.assign(character=Cast_In_df.character.str.split(',')).explode('character').reset_index(drop=True)
Cast_In_df.character= Cast_In_df.character.str.title()
Cast_In_df.character = Cast_In_df.character.str.replace('^ | $','',regex=True)
Cast_In_df = Cast_In_df.drop_duplicates(keep=False)
Cast_In_df.head(15)
#Cast_In_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Cast_In.tsv',index=False,na_rep=r'\N',sep='\t')

,tid,pid,character
0,tt0000001,nm1588970,SELF
1,tt0000005,nm0443482,BLACKSMITH
2,tt0000005,nm0653042,ASSISTANT
3,tt0000008,nm0653028,SNEEZING MAN
4,tt0000009,nm0063086,MISS GERALDINE HOLBROOK (MISS JERRY)
5,tt0000009,nm0183823,MR. HAMILTON
6,tt0000009,nm1309758,CHAUNCEY DEPEW - THE DIRECTOR OF THE NEW YORK ...
7,tt0000011,nm3692297,ACROBATS
8,tt0000012,nm2880396,SELF
9,tt0000012,nm9735580,SELF


In [ ]:
#Truncate list to tvSeries in the titles dataframe

q = """SELECT r.tid AS tid, pid, character
       FROM title_basics_df_2 t JOIN Cast_In_df r ON t.tconst = r.tid
      ;"""

Cast_In_df_2 = pysqldf(q)

In [ ]:
Cast_In_df_2.head(20)
Cast_In_df_2.describe()
Cast_In_df_2.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Cast_In.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
# Getting Basic Name data from Google Drive folder

name_basics = open("/content/gdrive/MyDrive/IMDb_Dataset/name_basics.tsv")
name_basics_df = pd.read_csv(name_basics, sep='\t', na_values='\\N',header=0)
#name_basics_df.head()
name_basics_df.dtypes

nconst                object
primaryName           object
birthYear            float64
deathYear            float64
primaryProfession     object
knownForTitles        object
dtype: object

In [ ]:
#Truncate list to pid in the PrincipalTitle dataframe

q = """SELECT *
      FROM name_basics_df
      WHERE nconst IN (SELECT DISTINCT pid FROM PrincipalTitle_df_2);"""

name_basics_df_2 = pysqldf(q)

In [ ]:
#Check truncated dataframe
name_basics_df_2.head(20)
name_basics_df_2.describe()

,birthYear,deathYear
count,154970.000000,37255.000000
mean,1959.963670,2001.641471
std,25.250599,16.948183
min,13.000000,1543.000000
25%,1943.000000,1992.000000
50%,1965.000000,2006.000000
75%,1979.000000,2015.000000
max,2021.000000,2022.000000


In [ ]:
#Cleaning Basic Name DF into IMDBPerson

IMDBPerson_df = name_basics_df_2.drop(columns={'primaryProfession','knownForTitles'})
IMDBPerson_df = IMDBPerson_df.rename(columns={'nconst':'pid','primaryName':'name'})
#IMDBPerson_df
IMDBPerson_df.head()

IMDBPerson_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/IMDBPerson.tsv',index=False,na_rep=r'\N',sep='\t')


In [ ]:
#Getting Basic Name into Professions 

Professions_df = name_basics_df_2[['nconst','primaryProfession']]
Professions_df = Professions_df.rename(columns={'nconst':'pid','primaryProfession':'profession'})
Professions_df = Professions_df.dropna()
Professions_df = Professions_df.assign(profession=Professions_df.profession.str.split(',')).explode('profession').reset_index(drop=True)
Professions_df.head()
Professions_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/Professions.tsv',index=False,na_rep=r'\N',sep='\t')

In [ ]:
#Getting Basic Name into KnownForTitles

KnownForTitles_df = name_basics_df_2[['nconst','knownForTitles']]
KnownForTitles_df = KnownForTitles_df.rename(columns={'nconst':'pid','knownForTitles':'knownFor'})
KnownForTitles_df = KnownForTitles_df.dropna()
KnownForTitles_df = KnownForTitles_df.assign(knownFor=KnownForTitles_df.knownFor.str.split(',')).explode('knownFor').reset_index(drop=True)
KnownForTitles_df.head()
KnownForTitles_df.to_csv('/content/gdrive/MyDrive/IMDb_Dataset/Processed/KnownForTitles.tsv',index=False,na_rep=r'\N',sep='\t')

#Section 3: Final List of Files Exported From Pre-Processing 


*   Additional Titles
*   Cast_In
*   Directors
*   Episode Titles
*   Genres
*   IMDB Person
*   Known For Titles
*   Principal Title
*   Professions
*   Ratings
*   Titles
*   Writers








